In [36]:
from cffi import FFI

In [37]:
ffi = FFI()
ffi.cdef("""
    long __stdcall IV_open();
    long __stdcall IV_close();
    long __stdcall IV_MaxDevices();
    long __stdcall IV_selectdevice(long *devnr);
    long __stdcall IV_getdevicestatus();
    long __stdcall IV_readSN(char *sntext);
    long __stdcall IV_connect(long* devconnect);
    long __stdcall IV_VersionHost(long *version);
    long __stdcall IV_VersionDll();
    long __stdcall IV_VersionCheck();
    long __stdcall IV_HostHandle();
    long __stdcall IV_VersionDllFile();
    long __stdcall IV_VersionDllFileStr();
    long __stdcall IV_SelectChannel(long *channel);
    
    long __stdcall IV_getcellstatus(long *devcellstatus);
    long __stdcall IV_setconnectionmode(long *value);
    long __stdcall IV_setcellon(long *cellon);
    long __stdcall IV_setpotential(double *value);
    long __stdcall IV_setpotentialWE2(double *value);
    long __stdcall IV_setcurrent(double *value);
    long __stdcall IV_getpotential(double *value);
    long __stdcall IV_setcurrentrange(long *value);
    long __stdcall IV_setcurrentrangeWE2(long *value);
    long __stdcall IV_getcurrent(double *value);
    long __stdcall IV_getcurrentWE2(double *value);
    long __stdcall IV_setfilter(long *value);
    long __stdcall IV_setstability(long *value);
    long __stdcall IV_setbistatmode(long *value);
    long __stdcall IV_setdac(long *channr, double *value);
    long __stdcall IV_getadc(long *channr, double *value);
    long __stdcall IV_setmuxchannel(long *value);
    long __stdcall IV_setdigout(long *value);
    long __stdcall IV_getdigin(long *value);
    long __stdcall IV_setfrequency(double *value);
    long __stdcall IV_setamplitude(double *value);
    long __stdcall IV_getcurrenttrace(long* npoints, double *rate, double *values);
    long __stdcall IV_getcurrentWE2trace(long* npoints, double *rate, double *values);
    long __stdcall IV_getpotentialtrace(long* npoints, double *rate, double *values);

    long __stdcall IV_we32setchannel(long *index);
    long __stdcall IV_we32setoffset(long *index, double *value);
    long __stdcall IV_we32setoffsets(long *nval, double *values);
    long __stdcall IV_we32getoffsets(long *nval, double *values);
    long __stdcall IV_we32readcurrents(double *values);

    long __stdcall IV_readmethod(char *fname);
    long __stdcall IV_savemethod(char *fname);
    long __stdcall IV_startmethod(char *fname);
    long __stdcall IV_abort();
    long __stdcall IV_savedata(char *fname);
    long __stdcall IV_setmethodparameter(char *parname, char *parvalue);
    long __stdcall IV_Ndatapoints(long *value);
    long __stdcall IV_getdata(long *pointnr, double *x, double *y, double *z);
    long __stdcall IV_getdatafromline(long *pointnr, long *scannr, double *x, double *y, double *z);

    long __stdcall IV_getDbFileName(char *fname);

    long __stdcall IV_StatusParGet(long *value);
    long __stdcall IV_StatusParSet(long *value);
""")

## Import dll

In [38]:
lib = ffi.dlopen("../pyvium_core/IVIUM_remdriver64.dll")

## IV_open()
###### no Iviumsoft --> -1
###### success --> 0
###### already open --> 0

In [39]:
def open_driver():
    return lib.IV_open()

In [40]:
open_driver()

0

## IV_close()

In [41]:
def close_driver():
    return lib.IV_close()

In [42]:
# iv_close()
# iv_open()

## IV_MaxDevices()

In [43]:
def get_max_device_number():
    return lib.IV_MaxDevices()

In [44]:
get_max_device_number()

24

## IV_readSN(char *sntext)
######  no IV_open() --> crash
######  no Iviumsoft --> -1 (rc=-1)
######  no device --> '' (rc=0)
######  device --> serialNum (rc=0)

In [45]:
def read_serial_number():
    serial_number_char_16 = ffi.new("char[]", 16)
    rc = lib.IV_readSN(serial_number_char_16)
   
    return rc, ffi.string(serial_number_char_16).decode("utf-8")

read_serial_number()

(0, 'B59340')

## IV_selectdevice(long *devnr)
######  no IV_open() --> crash
######  IV_open() --> integer values


In [46]:
def select_iviumsoft_instance(iviumsoft_instance_number):
    instance_number = ffi.new("long *", iviumsoft_instance_number)
    rc = lib.IV_selectdevice(instance_number)

    return rc,instance_number[0]

# iv_close()
# iv_open()
print(select_iviumsoft_instance(1))

(-2027738624, 1)


## IV_connect(long *devconnect)
######  no IV_open() --> crash
######  IV_open() --> with 1 connects device and with 0 disconnects it
###### software not running => -1
###### no device connected through usb => 1
###### device connected through usb    => 0 (device connected in iviumsoft)


In [47]:
## connection_status: 1 connect, 0 disconnect
def connect_device():
    connection_on = ffi.new("long *", 1)
    result_code = lib.IV_connect(connection_on)
    return lib.IV_connect(connection_on),connection_on[0]

def disconnect_device():
    connection_off = ffi.new("long *", 0)
    return lib.IV_connect(connection_off)

#open_driver()
# print(iv_select_iviumsoft_instance(1))
print(connect_device())
# print(2, iv_disconnect_device())

(-1, 1)


## IV_VersionDll()
######  no IV_open() --> crash
######  IV_open() --> returns the version for the driver's dll without points (200)


In [48]:
#open_driver()
def get_dll_version():
    return lib.IV_VersionDll()

# print(iv_get_dll_version())

## IV_VersionCheck()
######  no IV_open() --> crash
######  IV_open() --> returns 1 if the ivium software is running, -255 otherwise

In [49]:
#open_driver()
def is_iviumsoft_running():
    result_code = lib.IV_VersionCheck()
    return result_code

print(is_iviumsoft_running())

-256



## IV_getdevicestatus()

-1=no IviumSoft; 0=not connected; 1=available_idle; 2=available_busy; 3=no device available


In [50]:
def get_device_status():
    rc = lib.IV_getdevicestatus()
    print(rc)
    return lib.IV_getdevicestatus()
    
#get_device_status()


## Ndatapoints(int)

result_code => -1=no IviumSoft; 0=other case


In [51]:
open_driver()
def get_data_points_quantity():
    data_point = ffi.new("long *")

    result_code = lib.IV_Ndatapoints(data_point)

    return result_code, data_point[0]
    
#print(get_data_points_quantity())


## IV_getdata(int,d1,d2,d3)

result_code => -1=no IviumSoft; 0=other case

values => 0.0,0.0,0.0


In [52]:
#open_driver()
def get_data_point(data_point_index):
    selected_data_point_index = ffi.new("long *", data_point_index)
    measured_value1 = ffi.new("double *")
    measured_value2 = ffi.new("double *")
    measured_value3 = ffi.new("double *")

    result_code = lib.IV_getdata(selected_data_point_index, measured_value1, measured_value2, measured_value3)

    return result_code, measured_value1[0], measured_value2[0], measured_value3[0]
    
#print(get_data_point(2))


## IV_getdatafromline(int, int2, d1, d2, d3)

result_code => -1=no IviumSoft; 0=other case

values => 0.0,0.0,0.0


In [53]:
#open_driver()
def get_data_point_from_scan(data_point_index, scan_index):
    selected_data_point_index = ffi.new("long *", data_point_index)
    selected_line_index = ffi.new("long *", scan_index)
    measured_value1 = ffi.new("double *")
    measured_value2 = ffi.new("double *")
    measured_value3 = ffi.new("double *")

    result_code = lib.IV_getdatafromline(selected_data_point_index, selected_line_index, measured_value1, measured_value2, measured_value3)

    return result_code, measured_value1[0], measured_value2[0], measured_value3[0]
    
#print(get_data_point_from_scan(2, 3))

## IV_getcellstatus(long *devcellstatus)

In [54]:
def get_cell_status():
    cell_status_bits = ffi.new("long *")
    result_code = lib.IV_getcellstatus(cell_status_bits)
    cell_status_labels = []
    if result_code == 0:
        labels = ["I_ovl","", "Anin1_ovl","E_ovl","", "CellOff_button pressed", "Cell on"]
        for i, label in enumerate(labels,2):
            if cell_status_bits[0] & (1 << i) and label:
                cell_status_labels.append(label)
    return result_code, cell_status_labels
open_driver()
print(get_cell_status())
def IV_getcellstatus():
        '''Returns cell status labels
        ["I_ovl", "Anin1_ovl","E_ovl", "CellOff_button pressed", "Cell on"]'''
        cell_status_ptr = ffi.new("long *")
        result_code = lib.IV_getcellstatus(cell_status_ptr)
        return result_code,cell_status_ptr[0]
result_code, cell_status = IV_getcellstatus()
cell_status_labels = []
if result_code == 0:
    labels = ["I_ovl","", "Anin1_ovl","E_ovl","", "CellOff_button pressed", "Cell on"]
    for i, label in enumerate(labels,2):
        if cell_status & (1 << i) and label:
            cell_status_labels.append(label)
print(cell_status_labels)


(0, ['Cell on'])
['Cell on']


## IV_readmethod(char *)

In [55]:
open_driver()

def load_method(method_file_path):
    method_file_path_ptr = ffi.new("char []", method_file_path.encode("utf-8"))
    
    result_code = lib.IV_readmethod(method_file_path_ptr)

    return result_code, ffi.string(method_file_path_ptr).decode("utf-8")

# print(load_method("C:/Repos/test2.imf"))

## IV_savemethod(char *)

In [56]:
open_driver()

def save_method(method_file_path):
    method_file_path_ptr = ffi.new("char []", method_file_path.encode("utf-8"))
    
    result_code = lib.IV_savemethod(method_file_path_ptr)

    return result_code, ffi.string(method_file_path_ptr).decode("utf-8")

# print(save_method("C:/Repos/pepe.imf"))

## IV_startmethod(char *)

In [57]:
open_driver()

# def start_active_method():
#     method_mode_settings_path_ptr = ffi.new("char []", b'')
    
#     result_code = lib.IV_startmethod(method_mode_settings_path_ptr)

#     return result_code, ffi.string(method_mode_settings_path_ptr).decode("utf-8")

def start_method(method_file_path = ''):
    method_file_path_ptr = ffi.new("char []", method_file_path.encode("utf-8"))
    
    result_code = lib.IV_startmethod(method_file_path_ptr)

    return result_code, ffi.string(method_file_path_ptr).decode("utf-8")

# print(start_method())
# print(start_method("test2.imf"))

## IV_savedata(char *)

In [58]:
open_driver()

def save_method_data(method_data_file_path):
    method_data_file_path_ptr = ffi.new("char []", method_data_file_path.encode("utf-8"))
    
    result_code = lib.IV_savedata(method_data_file_path_ptr)

    return result_code, ffi.string(method_data_file_path_ptr).decode("utf-8")

# print(save_method_data("results2.idf"))

## IV_abort(char *)

In [59]:
open_driver()

def abort_method():
    result_code = lib.IV_abort()

    return result_code

#print(abort_method())

## IV_setmethodparameter(char *, char *)

In [60]:
open_driver()

def set_method_parameter_value(parameter_name, parameter_value):
    parameter_name_ptr = ffi.new("char []", parameter_name.encode("utf-8"))
    parameter_value_ptr = ffi.new("char []", parameter_value.encode("utf-8"))
    
    result_code = lib.IV_setmethodparameter(parameter_name_ptr, parameter_value_ptr)

    return result_code

#print(set_method_parameter_value("Mode", "HiSpeed2"))
#print(set_method_parameter_value("Title", "New Scan1"))
#print(set_method_parameter_value("+Redefine filename", "Checked"))

close_driver()

0

## IV_setconnectionmode(long *value)

In [61]:
# Select configuration, 0=off; 1=EStat4EL(default), 2=EStat2EL,3=EstatDummy1,4=EStatDummy2,5=EstatDummy3,6=EstatDummy4, 7=Istat4EL, 8=Istat2EL, 9=IstatDummy (missing!!!), 10=BiStat4EL, 11=BiStat2EL


def set_connection_mode(connection_mode_number):
    connection_mode_number_ptr = ffi.new("long *",connection_mode_number)
    result_code = lib.IV_setconnectionmode(connection_mode_number_ptr)

    return result_code

#set_connection_mode(2)

## IV_getcurrenttrace(long* npoints, double *rate, double *values)

In [62]:
# npoints<=256, interval: 10us to 20ms

# revienta para valores (3, 10)
def get_current_trace(points_quantity, interval_rate):
    points_quantity_ptr = ffi.new("long *",points_quantity)
    interval_rate_ptr = ffi.new("double *",interval_rate)
    result_ptr = ffi.new("double *")

    result_code = lib.IV_getcurrenttrace(points_quantity_ptr, interval_rate_ptr, result_ptr)

    return result_code, result_ptr[0]

#open_driver()
#get_current_trace(2, 10)
#close_driver()

## IV_getcurrentWE2trace(long* npoints, double *rate, double *values)

In [63]:
# npoints<=256, interval: 10us to 20ms
def get_current_we2_trace(points_quantity, interval_rate):
    points_quantity_ptr = ffi.new("long *",points_quantity)
    interval_rate_ptr = ffi.new("double *",interval_rate)
    result_ptr = ffi.new("double *")

    result_code = lib.IV_getcurrentWE2trace(points_quantity_ptr, interval_rate_ptr, result_ptr)

    return result_code, result_ptr[0]

# open_driver()
# get_current_we2_trace(1, 1)
# close_driver()

## IV_getpotentialtrace(long* npoints, double *rate, double *values)

In [64]:
# npoints<=256, interval: 10us to 20ms
def get_current_potencial_trace(points_quantity, interval_rate):
    points_quantity_ptr = ffi.new("long *",points_quantity)
    interval_rate_ptr = ffi.new("double *",interval_rate)
    result_ptr = ffi.new("double *")

    result_code = lib.IV_getpotentialtrace(points_quantity_ptr, interval_rate_ptr, result_ptr)

    return result_code, result_ptr[0]

# open_driver()
# get_current_potencial_trace(1, 1)
# close_driver()

##IV_setcellon

In [69]:
def IV_setcellon(cell_on_mode_number):
        '''Set cell on off to close cell relais (0=off;1=on)'''
        cell_on_mode_number_ptr = ffi.new("long *", cell_on_mode_number)
        result_code = lib.IV_setconnectionmode(
            cell_on_mode_number_ptr)

        return result_code
IV_setcellon(4)

0